In [18]:
import pandas as pd
import os
import json
from pyecharts.charts import Graph
from pyecharts import options as opts
from pyecharts.globals import ThemeType
import re
import difflib
from pyecharts.commons.utils import JsCode
from bs4 import BeautifulSoup as bs
import base64
import numpy as np
import gzip
import io
from PIL import Image

In [19]:
def convert_png_to_webp(src_directory):
    for root, dirs, files in os.walk(src_directory):
        for file in files:
            if file.endswith('.png'):
                png_file_path = os.path.join(root, file)
                webp_file_path = os.path.splitext(png_file_path)[0] + '.webp'

                # Create the directory for the webp file if it doesn't exist
                os.makedirs(os.path.dirname(webp_file_path), exist_ok=True)

                # Convert PNG to WEBP
                with Image.open(png_file_path) as img:
                    img.save(webp_file_path, 'WEBP')
    print("done")


In [28]:
GTNH_version = "280"
webp_buffer = io.BytesIO()


convert_png_to_webp(f'quests_icons/{GTNH_version}/')
if not os.path.exists("json"):
    os.mkdir("json")
if not os.path.exists(f"json/{GTNH_version}"):
    os.mkdir(f"json/{GTNH_version}")
if not os.path.exists(f"json/{GTNH_version}/quests_icons/"):
    os.mkdir(f"json/{GTNH_version}/quests_icons/")
if not os.path.exists(f"json/{GTNH_version}/quests_icons/QuestIcon/"):
    os.mkdir(f"json/{GTNH_version}/quests_icons/QuestIcon/")
if not os.path.exists("json"):
        os.mkdir("json")

need_to_replace = ["§%n", "§\n"]

quest_map = {}
quest_line_map = {}
quest_id_set = set()
line_id_set = set()
quest_zh_map = {}
quest_zh_line_map = {}
with open(f"lang/{GTNH_version}/zh_CN.lang", "r", encoding="utf-8") as f:
    for line in f.readlines():
        if "betterquesting.questline" in line and "=" in line:
            one_line = line.split("=")
            quest_line_id = one_line[0].split(".")[-2]
            if "name" in one_line[0]:
                quest_zh_line_map[quest_line_id + "name"] = one_line[1]
            elif "desc" in one_line[0]:
                quest_zh_line_map[quest_line_id + "desc"] = one_line[1]
            line_id_set.add(quest_line_id)
        elif "betterquesting.quest" in line and "=" in line:
            one_line = line.split("=", 1)
            quest_id = one_line[0].split(".")[-2]
            if "name" in one_line[0]:
                quest_zh_map[quest_id + "name"] = one_line[1]
            elif "desc" in one_line[0]:
                quest_zh_map[quest_id + "desc"] = one_line[1]
            quest_id_set.add(quest_id)

quest_name_df = pd.DataFrame(columns=["id", "name", "desc", "zh_name", "zh_desc"])
for i in quest_id_set:
    name = quest_map[i + "name"].strip() if (i + "name") in quest_map else ""
    desc = quest_map[i + "desc"].strip() if (i + "desc") in quest_map else ""
    zh_name = quest_zh_map[i + "name"].strip() if (i +
                                                   "name") in quest_zh_map else ""
    zh_desc = quest_zh_map[i + "desc"].strip()if (i +
                                                  "desc") in quest_zh_map else ""

    quest_name_df.loc[len(quest_name_df)] = [i.replace(
        "quest", ""), name, desc, zh_name, zh_desc]

quest_line_name_df = pd.DataFrame(
    columns=["id", "name", "desc", "zh_name", "zh_desc"])
for i in line_id_set:
    name = quest_line_map[i + "name"].strip() if (i +
                                                  "name") in quest_line_map else ""
    desc = quest_line_map[i + "desc"].strip() if (i +
                                                  "desc") in quest_line_map else ""
    zh_name = quest_zh_line_map[i +
                                "name"].strip() if (i + "name") in quest_zh_line_map else ""
    zh_desc = quest_zh_line_map[i +
                                "desc"].strip()if (i + "desc") in quest_zh_line_map else ""
    quest_line_name_df.loc[len(quest_line_name_df)] = [i.replace(
        "line", ""), name, desc, zh_name, zh_desc]

with open(f"DefaultQuests/{GTNH_version}/QuestLinesOrder.txt", "r") as f:
    en_quest_line = f.readlines()

quest_line_json = []
quest_line_icons_dict = {}
for l in en_quest_line:
    quest_line_id = re.sub("[^a-zA-Z0-9_]", "", l.split(":")[0])
    quest_line_name = l.split(":")[1].strip()
    line = quest_line_name_df[quest_line_name_df["id"] == quest_line_id]

    shorter_name = re.sub("[^a-zA-Z0-9]", "", quest_line_name)[:16]
    quest_line_json.append({
        "title": quest_line_name,
        "title_zh": line["zh_name"].values[0],
        "quest": shorter_name,
    })

    with open(f"quests_icons/{GTNH_version}/QuestLineIcon/{shorter_name}.webp", "rb") as img:
        quest_line_icons_dict[shorter_name + ".webp"] = base64.b64encode(img.read()).decode()
    # with Image.open(f"quests_icons/{GTNH_version}/QuestLineIcon/{shorter_name}.png") as img:
    #     img.save(webp_buffer, format="webp")
    #     quest_line_icons_dict[shorter_name + ".webp"] = base64.b64encode(webp_buffer.getvalue()).decode()
    #     webp_buffer.truncate(0)


with open(f"json/{GTNH_version}/quest_line.json", "w", encoding="utf8") as f:
    f.write(str(quest_line_json).replace("'", "\""))

with open(f"json/{GTNH_version}/quests_icons/QuestLineIcon.gtbl", "wb") as f:
    f.write(gzip.compress(json.dumps(
        quest_line_icons_dict).encode("utf-8")))

all_icon_map = {"QuestLineIcon": [i["quest"] for i in quest_line_json]}


for ver in ["zh", "en"]:
    df = pd.DataFrame(columns=["quest_file_name", "quest_id",
                               "quest_name", "quest_desc", "quest_icon", "pre_quests", "quest_name_with_format", "is_main"])
    for quest_line in os.listdir(f"DefaultQuests/{GTNH_version}/Quests/"):
        for quest in os.listdir(f"DefaultQuests/{GTNH_version}/Quests/{quest_line}/"):
            que = json.load(
                open(f"DefaultQuests/{GTNH_version}/Quests/{quest_line}/{quest}", "r", encoding="utf-8"))

            quest_high_id = que["questIDHigh:4"] if "questIDHigh:4" in que else 0
            quest_low_id = que["questIDLow:4"]
            quest_name = que["properties:10"]["betterquesting:10"]["name:8"]
            quest_desc = que["properties:10"]["betterquesting:10"]["desc:8"]
            icon_name = str(quest_low_id) + ".webp"
            is_main = que["properties:10"]["betterquesting:10"]["isMain:1"]

            pre_quests = []
            for i in que["preRequisites:9"]:
                pre_quests.append(que["preRequisites:9"][i]["questIDLow:4"])

            str_id = base64.urlsafe_b64encode(bytes.fromhex((quest_high_id.to_bytes(length=8, signed=True, byteorder="big") +
                                                             quest_low_id.to_bytes(length=8, signed=True, byteorder="big")).hex())).decode("utf-8").replace("==", "")
            if ver == "zh":
                line = quest_name_df[quest_name_df["id"] == str_id]
                if len(line) != 0:
                    quest_name = line["zh_name"].values[0]
                    quest_desc = line["zh_desc"].values[0]

            for s in need_to_replace:
                quest_name = quest_name.replace(s, "")
                quest_desc = quest_desc.replace(s, "")

            quest_desc = quest_desc.replace("%n", "<br/>")
            quest_desc = quest_desc.replace("\n", "<br/>")

            df.loc[len(df)] = [quest, quest_low_id, re.sub("(§.)", "", quest_name.strip()),
                               quest_desc, icon_name, pre_quests, quest_name, is_main]

    repeats = list(set(df[df["quest_name"].duplicated()]["quest_name"]))
    for word in repeats:
        counter = 1
        for i in df[df["quest_name"] == word].sort_values("quest_id").index:
            df.loc[i, "quest_name"] = df.loc[i, "quest_name"] + str(counter)
            counter += 1

    # ========================================== #

    whole_json = {}

    for questline in os.listdir(f"DefaultQuests/{GTNH_version}/QuestLines"):
        nodes = []
        links = []
        max_size = 0
        node_size = []
        fake_index = 0
        quests_icons_map = {}
        icon_list = []
        for quest in os.listdir(f"DefaultQuests/{GTNH_version}/QuestLines/{questline}"):
            if quest == "QuestLine.json":
                continue

            que = json.load(
                open(f"DefaultQuests/{GTNH_version}/QuestLines/{questline}/{quest}", "r", encoding="utf-8"))
            line = df[df["quest_id"] == que["questIDLow:4"]]
            desc = line["quest_desc"].values[0]

            if ver == "zh":
                with open(f"quests_icons/{GTNH_version}/QuestIcon/{line['quest_icon'].values[0]}", "rb") as img:
                    quests_icons_map[line['quest_icon'].values[0]
                                     ] = base64.b64encode(img.read()).decode()
                icon_list.append(line['quest_icon'].values[0].split(".")[0])

            name = line["quest_name"].values[0]

            if "questIDHigh:4" in que:
                high_id = que["questIDHigh:4"]
            else:
                high_id = 0
            low_id = que["questIDLow:4"]

            quest_id_str = base64.urlsafe_b64encode(bytes.fromhex((high_id.to_bytes(
                length=8, signed=True, byteorder="big") + low_id.to_bytes(length=8, signed=True, byteorder="big")).hex())).decode("utf-8")

            title = line["quest_name_with_format"].values[0]

            is_main = line["is_main"].values[0]

            new_node = {"name": name,
                        "symbolSize": int(que["sizeX:3"]),
                        "symbol": "",
                        "x": (que["x:3"] + int(que["sizeX:3"]) / 2),
                        "y": (que["y:3"] + int(que["sizeX:3"]) / 2),
                        "data": desc,
                        "quest_id": quest_id_str,
                        "title": title,
                        "is_main": 1 if is_main else 0,
                        "tooltip": {
                            "show": True,
                        },

                        }

            node_size.append(new_node["symbolSize"] * 1.3)

            nodes.append(new_node)
            fake_index += 1
            pre_quest_names = []
            pre_quests = line["pre_quests"].values[0]
            for quest_id in pre_quests:
                pre_que = df[df["quest_id"] == quest_id]
                pre_name = pre_que["quest_name"].values[0]
                pre_quest_names.append(pre_name)

                temp_link = {"source": pre_name,
                             "target": name,
                             "symbol": ["none", "arrow"], "lineStyle": {"width": 2}, "symbolSize": [0, 10]}
                if int(pre_que["is_main"].iloc[0]) == int(is_main) == 1:
                    temp_link["lineStyle"] = {"color": "#00c800", "width": 4}
                links.append(temp_link)

            if ver == "zh":
                new_node["tooltip"] = f"<strong>{name}</strong>"
                if len(pre_quest_names) > 0:
                    new_node["tooltip"] += "<br/>前置任务需求:<br/>" + \
                        "<br/>".join(pre_quest_names)
            elif ver == "en":
                new_node["tooltip"] = f"<strong>{name}</strong>"
                if len(pre_quest_names) > 0:
                    new_node["tooltip"] += "<br/>Prequest requests:<br/>" + \
                        "<br/>".join(pre_quest_names)

        # 调整所有任务图标大小
        avg = sum(node_size) / len(node_size)
        if sum(node_size) <= 1000:
            multi = 1.8
        else:
            multi = 37 / avg
        for node in nodes:
            node["symbolSize"] = round(node["symbolSize"] * multi, 2)

        print(
            f"questline {questline.split("-", 1)[0]},\n there has {len(node_size)} nodes, node size sum {"%.2f" % sum(node_size)}, avg size is {"%.2f" % avg}, multi is{"%.2f" % multi}")

        if ver == "zh":
            with open(f"json/{GTNH_version}/quests_icons/QuestIcon/{questline.split("-", 1)[0]}.gtbl", "wb") as f:
                f.write(gzip.compress(json.dumps(
                    quests_icons_map).encode("utf-8")))
            all_icon_map[f"QuestIcon/{questline.split("-", 1)[0]}"] = icon_list

        g = (
            Graph()
            .add("", nodes, links)
        )
        series = g.options["series"][0]
        whole_json[questline.split(
            '-')[0]] = {"data": series["data"], "links": series["links"]}

    with open(f"json/{GTNH_version}/quest_json{"" if ver == "zh" else "_en"}.json", "w", encoding="utf-8") as quest_json:
        json.dump(whole_json, quest_json,
                  separators=(',', ':'), ensure_ascii=False)
    if ver == "zh":
        with open(f"json/{GTNH_version}/quests_icons.json", "w", encoding="utf8") as f:
            json.dump(all_icon_map, f)

done
questline AdeptThaumaturgy,
 there has 77 nodes, node size sum 2402.40, avg size is 31.20, multi is1.19
questline AndSoItBegins,
 there has 45 nodes, node size sum 1872.00, avg size is 41.60, multi is0.89
questline AppliedEnergisti,
 there has 192 nodes, node size sum 6084.00, avg size is 31.69, multi is1.17
questline BasicAutomation,
 there has 35 nodes, node size sum 1092.00, avg size is 31.20, multi is1.19
questline BeeBreeding,
 there has 169 nodes, node size sum 5678.40, avg size is 33.60, multi is1.10
questline BiofortheMasses,
 there has 49 nodes, node size sum 1840.80, avg size is 37.57, multi is0.98
questline BuildingBetterBa,
 there has 94 nodes, node size sum 2995.20, avg size is 31.86, multi is1.16
questline CoinsCoinsCoins,
 there has 177 nodes, node size sum 5522.40, avg size is 31.20, multi is1.19
questline EndgameGoals,
 there has 19 nodes, node size sum 592.80, avg size is 31.20, multi is1.80
questline FishingFarmingCo,
 there has 107 nodes, node size sum 3338.40,

In [34]:
quest_name_df[quest_name_df["zh_name"].str.contains("量子计算机")]["zh_desc"].iloc[0]

'量子计算机(QC)是一种极其复杂的机器,主要用于为研究站和其他一些多方块机器生成算力.在使用之前,你需要了解一些事情.%n%n§l建造指示§r. 与装配线一样,量子计算机也是以一系列切片的形式构建的.第一片和最后一片是基础舱室的唯一的有效位置.中间的切片主要用于安装计算机机架.多块结构全息投影仪会显示最小长度的量子计算机,但同时持有多个投影仪会显示/构建更长的版本,最大长度可达16.在计算机散热风扇前还有一个强制性的空气块.%n%n§l输出§r. 对于某些配方来说,计算不过是一个整数输入--就像EU一样.可能存在最低吞吐量要求、需要达到的阈值或两者兼而有之(NEI会告诉你一切).量子计算机每秒都会产生算力,由光传输连接器传输,由光接收连接器接收,并由需要它的机器或流程“消耗”.没有方法可以存储算力,同一时间只能有一个传输或接收舱口连接到同一根光纤电缆上.如果绝对有必要,带有QoS的网络交换机可以拆分/连接,但要知道,它的运行既需要电力,也需要维护.%n%n机器之间可以使用另一组光传输和光接收连接器将附加的量子计算机串联起来,但它们不能共用同一个光传输连接器.%n%n§l未定元§r. 量子计算机有一个独特的维护要求,只有通过平衡未定元解析器内的薛定谔矩阵才能纠正--这是一个额外的仓室,必须安装在机器的第一或最后一片上.从基本的“未定元解析器”开始,不过后面还可以解锁两种升级版的仓室,这两种先进未定元解析器会降低谜题的难度,或者直接杀死谜题?%n%n未定元解析器的界面中间是薛定谔矩阵,左右两边各有8个按钮.点击一个按钮,再点击另一个按钮,就能切换矩阵中相应两个单元格的状态.这意味着切换它们的闪烁速度(在基础未定元解析器中)或颜色(在未定元解析器X中).谜题的目标是平衡矩阵中发光LED灯周围的状态,使所有的LED灯都变成绿色.%n%n§l可调参数§r. 在控制器的GUI中,有两个可配置的参数,即超频比和过压比,它们可以改变计算和发热的方式.超频比越高,计算量越大,过压比越高,发热量越大.单独来看,这将鼓励玩家尽可能提高超频,尽可能降低过电压,但这两个参数之间的巨大差异会对计算产生影响.此外,过压比小于1会导致QC随机降低算力,降低的幅度与它与1的距离成正比.我们强烈建议您仔细研究这些参数,因为所有最佳设置都会这样做.%n%n请参阅相邻的任务,了解有关计算机机架和生成算力

In [ ]:
ver = "en"

GTNH_version = "280"

df = pd.DataFrame(columns=["quest_file_name", "quest_id",
                  "quest_name", "quest_desc", "quest_icon", "pre_quests", "quest_name_with_format", "is_main"])
for quest_line in os.listdir("DefaultQuests/Quests/"):
    for quest in os.listdir(f"DefaultQuests/Quests/{quest_line}/"):
        que = json.load(
            open(f"DefaultQuests/Quests/{quest_line}/{quest}", "r", encoding="utf-8"))

        quest_high_id = que["questIDHigh:4"] if "questIDHigh:4" in que else 0
        quest_low_id = que["questIDLow:4"]
        quest_name = que["properties:10"]["betterquesting:10"]["name:8"]
        quest_desc = que["properties:10"]["betterquesting:10"]["desc:8"]
        icon_name = str(quest_low_id) + ".png"
        is_main = que["properties:10"]["betterquesting:10"]["isMain:1"]
        
        pre_quests = []
        for i in que["preRequisites:9"]:
            pre_quests.append(que["preRequisites:9"][i]["questIDLow:4"])

        # if f"quest{quest_id}name" in quest_zh_map:
        #     quest_name = quest_zh_map[f"quest{quest_id}name"]
        # if f"quest{quest_id}desc" in quest_zh_map:
        #     quest_desc = quest_zh_map[f"quest{quest_id}desc"]
        
        str_id = base64.urlsafe_b64encode(bytes.fromhex((quest_high_id.to_bytes(length=8, signed=True, byteorder="big") +
                                          quest_low_id.to_bytes(length=8, signed=True, byteorder="big")).hex())).decode("utf-8").replace("==", "")
        if ver == "zh":
            line = quest_name_df[quest_name_df["id"] == str_id]
            if len(line) != 0:
                quest_name = line["zh_name"].values[0]
                quest_desc = line["zh_desc"].values[0]

        for s in need_to_replace:
            quest_name = quest_name.replace(s, "")
            quest_desc = quest_desc.replace(s, "")

        quest_desc = quest_desc.replace("%n", "<br/>")
        quest_desc = quest_desc.replace("\n", "<br/>")

        # quest_name = re.sub("(§.)", "", quest_name.strip())
        # quest_desc = re.sub("(§.)", "", quest_desc.strip())


        df.loc[len(df)] = [quest, quest_low_id, re.sub("(§.)", "", quest_name.strip()),
                           quest_desc, icon_name, pre_quests, quest_name, is_main]


repeats = list(set(df[df["quest_name"].duplicated()]["quest_name"]))
for word in repeats:
    counter = 1
    for i in df[df["quest_name"] == word].sort_values("quest_id").index:
        df.loc[i, "quest_name"] = df.loc[i, "quest_name"] + str(counter)
        counter += 1

# ========================================== #

if_json = True
whole_json = json.loads("{}")

for questline in os.listdir("DefaultQuests/QuestLines"):
    nodes = []
    links = []
    max_size = 0
    node_size = []
    fake_index = 0
    quests_icons_map = {}
    icon_list = []
    for quest in os.listdir(f"DefaultQuests/QuestLines/{questline}"):
        if quest == "QuestLine.json":
            continue

        que = json.load(
            open(f"DefaultQuests/QuestLines/{questline}/{quest}", "r", encoding="utf-8"))
        line = df[df["quest_id"] == que["questIDLow:4"]]

        desc = line["quest_desc"].values[0]

        temp_list = []
        for i in range(0, len(desc), 50):
            temp_list.append(desc[i: i + 50])

        # path = f"image://version/{GTNH_version}/{line['quest_icon'].values[0]}"
        with open(f"quests_icons/QuestIcon/{line['quest_icon'].values[0]}", "rb") as f:
            quests_icons_map[line['quest_icon'].values[0]] = base64.b64encode(f.read()).decode()
        icon_list.append(line['quest_icon'].values[0].split(".")[0])

        name = line["quest_name"].values[0]

        if "questIDHigh:4" in que:
            high_id = que["questIDHigh:4"]
        else:
            high_id = 0
        low_id = que["questIDLow:4"]

        quest_id_str = base64.urlsafe_b64encode(bytes.fromhex((high_id.to_bytes(
            length=8, signed=True, byteorder="big") + low_id.to_bytes(length=8, signed=True, byteorder="big")).hex())).decode("utf-8")

        title = line["quest_name_with_format"].values[0]

        is_main = line["is_main"].values[0]

        new_node = {"name": name,
                    "symbolSize": int(que["sizeX:3"]),
                    "symbol": "",
                    "x": (que["x:3"] + int(que["sizeX:3"]) / 2),
                    "y": (que["y:3"] + int(que["sizeX:3"]) / 2),
                    "data": desc,
                    "quest_id": quest_id_str,
                    "title": title,
                    "is_main": 1 if is_main else 0,
                    "tooltip": {
                        "show": True,
                    },
                    }

        node_size.append(new_node["symbolSize"] * 1.3)

        nodes.append(new_node)
        fake_index += 1
        pre_quest_names = []
        pre_quests = line["pre_quests"].values[0]
        for quest_id in pre_quests:
            pre_que = df[df["quest_id"] == quest_id]
            pre_name = pre_que["quest_name"].values[0]
            pre_quest_names.append(pre_name)

            temp_link = {"source": pre_name,
                         "target": name,
                         "symbol": ["none", "arrow"]}
            if int(pre_que["is_main"].iloc[0]) == int(is_main) == 1:
                temp_link["lineStyle"] = {"color": "#00c800", "width": 3}
            links.append(temp_link)

        if ver == "zh":
            new_node["tooltip"] = "<strong>" + name + \
                "</strong><br/>前置任务需求:<br/>" + "<br/>".join(pre_quest_names)
        elif ver == "en":
            new_node["tooltip"] = "<strong>" + name + \
                "</strong><br/>Prequest requests:<br/>" + \
                "<br/>".join(pre_quest_names)

    avg = sum(node_size) / len(node_size)
    if sum(node_size) <= 1000:
        multi = 1.8
    else:
        multi = 37 / avg

    for node in nodes:
        node["symbolSize"] *= multi

    print(
        f"questline {questline.split("-", 1)[0]},\n there has {len(node_size)} nodes, node size sum {"%.2f" % sum(node_size)}, avg size is {"%.2f" % avg}, multi is{"%.2f" % multi}")

    # with open(f"json/quests_icons/QuestIcon/{questline.split("-", 1)[0]}.json", "w", encoding="utf8") as f:
    #     json.dump(quests_icons_map, f, indent=4)
    with open(f"json/quests_icons/QuestIcon/{questline.split("-", 1)[0]}.gtbl", "wb") as f:
        f.write(gzip.compress(json.dumps(
            quests_icons_map, indent=4).encode("utf-8")))
    all_icon_map[f"QuestIcon/{questline.split("-", 1)[0]}"] = icon_list

    g = (
        Graph(init_opts=opts.InitOpts(width="100%",
              height="900%", bg_color="#f5f0d3", chart_id="this_chart", ))
        .add("", nodes, links, is_draggable=False, edge_symbol=['circle', 'arrow'], repulsion=0, label_opts=opts.LabelOpts(position="up", is_show=False), gravity=0, layout="none", edge_symbol_size=10)
    )
    if if_json:
        json_str = g.dump_options_with_quotes()
        series = json.loads(json_str)["series"][0]
        whole_json[questline.split(
            '-')[0]] = {"data": series["data"], "links": series["links"]}
    else:
        g.render(f"html/{questline.split('-')[0]}.html")

if if_json:
    with open(f"json/quest_json{"" if ver == "zh" else "_en"}.json", "w", encoding="utf-8") as quest_json:
        json.dump(whole_json, quest_json, indent=4,
                  separators=(',', ':'), ensure_ascii=False)
    with open("json/quests_icons.json", "w", encoding="utf8") as f:
        json.dump(all_icon_map, f, indent=4)
else:
    for html in os.listdir("html"):
        if not "html" in html:
            continue
        with open(f"html/{html}", encoding="utf-8") as page:
            a = bs(page)
        os.remove(f"html/{html}")
        a = a.find_all("script")[1]
        with open(f"js/{html.replace('.html', '.js')}", "w", encoding="utf-8") as js:
            js.write(a.text)

In [ ]:
gzip.compress(json.dumps(quests_icons_map, indent=4).encode("utf-8"))

In [ ]:
with open(r"E:\Better_Online_QuestBook\bin\version\280\quests_icons\QuestIcon\WorkingwithOil.gtbl", "br") as f:
    print(f.read().__len__())

In [ ]:
json.dumps(quests_icons_map, indent=4).encode("utf-8").__len__()

In [ ]:
# for sql process

def get_str_prequest_id(quests: list):
    temp = []
    for quest in quests:
        temp.append(df[df["quest_id"] == quest]["quest_id_str"].values[0])
    return temp

df["quest_id_str"] = df["quest_file_name"].apply(
    lambda x: x.split("-", 1)[1].replace(".json", ""))
df["pre_quests_str"] = df["pre_quests"].apply(lambda x: get_str_prequest_id(x))

with open("GTNHquest.sql", mode="w", encoding="utf-8") as sql:
    sql.write("# 赞美钛管道大人\n")
    sql.write("""
DROP TABLE IF EXISTS `quest`;
CREATE TABLE `quest`(
    `id` INT NOT NULL PRIMARY KEY AUTO_INCREMENT,
    `quest_id` TEXT NOT NULL,
    `quest_name` TEXT NOT NULL,
    `quest_desc` TEXT NOT NULL,
    `pre_quests` TEXT NOT NULL
);

""")
    for index in range(len(df)):
        line = df.iloc[index]
        sql.write(f"""INSERT INTO `quest` VALUES (NULL, '{line["quest_id_str"]}', '{line["quest_name"].replace("'", "''")}', '{re.sub(r"(§.)?(\[/?(note|url|warn)/?\])?", "", line['quest_desc'].replace('<br/>', '\\n').strip()).replace("'", "''")}', '{', '.join(line["pre_quests_str"])}');
""")

In [ ]:
a = "_N4B3gVMTZOoUdf9--7aPQ=="

print("高64位", bin(int(base64.urlsafe_b64decode(a.encode("utf-8")).hex()[:-16], 16))[2:])
print("低64位", bin(int(base64.urlsafe_b64decode(a.encode("utf-8")).hex()[-16:], 16))[2:])

In [ ]:
print("低64位", bin(-6318031316370662851 & 0xFFFFFFFFFFFFFFFF)[2:])

In [ ]:
base64.urlsafe_b64encode(bytes.fromhex((quest_high_id.to_bytes(length=8, signed=True, byteorder="big") +
                                        quest_low_id.to_bytes(length=8, signed=True, byteorder="big")).hex())).decode("utf-8").replace("==", "")

In [ ]:
quest_line_order = []
for line in open("DefaultQuests/QuestLinesOrder.txt").readlines():
    quest_line_order.append(line.split(": ")[1].strip())

In [ ]:
quest_line_order

In [ ]:
counter = 0
with open("lang/en_US.lang", "r", encoding="utf-8") as f:
    for line in f.readlines():
        if "=" in line:
            counter += 1
counter

In [ ]:
counter = 0
with open("lang/zh_CN.lang", "r", encoding="utf-8") as f:
    for line in f.readlines():
        if "=" in line:
            counter += 1
counter

In [ ]:
int(base64.b64decode("LIZCdBLfQquQpULGmFDixw==".encode("ascii")).hex(), 16)

In [ ]:
base64.b64encode(bytes.fromhex("0" * (32-3) + hex(260)[2:])).decode("utf-8")

In [ ]:
base64.b64encode(bytes.fromhex((que["questIDHigh:4"].to_bytes(length=8, signed=True) +
                                que["questIDLow:4"].to_bytes(length=8, signed=True)).hex())).decode("utf-8")

In [ ]:
"LIZCdBLfQquQpULGmFDixw=="

In [ ]:
temp_str = "4096kEssentiaSto-G5g_5_24RS6sqhmjFEbf8A==.json".split("-", 1)[
    1].replace(".json", "")
re.sub("[^a-zA-Z0-9/-/_]", "", temp_str)

In [ ]:
temp_str = "欢迎来到TecTech!这里有你毕业所需的真正的巅峰科技.但是,学习TecTech常常让人摸不着头脑,所以仔细听.%n%n首先你需要制造所需机器的多种机械方块:超能机械方块,电子计算机机械方块,高级计算机方块和计算机散热器.你需要大量这些机械方块才能继续前进.%n%n我们列举出来的只是举个例子,实际需要的只会更多、更多、更多.%n%n使用光缆将机器连接起来. 确保它们已被涂色,同时光缆连接到的方块也要被涂色.%n%n*投影仪将会显示完整机器的全息图像,左下角的聊天栏会显示可变方块的注释. 如果堆叠更多投影仪的话,你就可以看到较大版本的全息图(比如量子计算机).%n%n§4译者注:§lTecTech多方块机器的控制器GUI界面的闪电或是电源图标亮起时,不要破坏任何结构方块,否则会引发爆炸!运行过程中更不能破坏结构方块!§r"

In [ ]:
temp_q = ""
flag = False
code = []
for i in re.split("(§.)", temp_str):
    if len(i) == 0:
        continue
    if "§" in i:
        flag = True
        if "r" in i:
            flag = False
            print(temp_q)
            print(code)
            code = []
        else:
            code.append(i)
    else:
        if flag is False:
            print(i)
        else:
            temp_q += i

In [ ]:
re.split("(§.)", temp_str)

In [ ]:
def process(temp_str: str, type: list[str]):
    ...